In [1]:
!python -m spacy download fr_core_news_sm

✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [2]:
#Importing required packages
import pandas as pd
import seaborn as sns
import nltk
import matplotlib.pyplot as plt
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import numpy as np
import gensim
from sklearn import datasets
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.utils.multiclass import unique_labels
import string

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
#Import data
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
Sentences = "https://raw.githubusercontent.com/TetraFaal/Big-Scale-Analytics-2021-Tesla/main/Data/Sentences-Tesla.csv"
Sentences = pd.read_csv(Sentences)
Sentences["Text"] = Sentences['Text'].apply(remove_punctuations)

Cognates = "https://raw.githubusercontent.com/TetraFaal/Big-Scale-Analytics-2021-Tesla/main/Data/Cognates.csv"
Cognates = pd.read_csv(Cognates)

CognatesGlossary = "https://raw.githubusercontent.com/TetraFaal/Big-Scale-Analytics-2021-Tesla/main/Data/glossaryCognates.csv"
CognatesGlossary = pd.read_csv(CognatesGlossary)


In [5]:
Cognates.head()

,Cognates
0,abattoir
1,abdication
2,abdominal
3,aberrant
4,aberration


In [6]:
CognatesGlossary.head()

,CognatesEN,CognatesFR
0,absolutely,absolument
1,abundance,abondance
2,abuse,abuser
3,accelerate (to),accélérer
4,accentuate (to),accentuer


In [7]:
#Create a cognates list and delete duplicates
ListCo = []
for cognate  in CognatesGlossary.CognatesFR:
  ListCo.append(cognate)
for cognate  in Cognates.Cognates:
  ListCo.append(cognate)
ListCognates = list(set(ListCo))
print(ListCognates)
print(len(ListCognates))

['jockey', 'hibiscus', 'concave', 'job', 'millet', 'apartheid', 'macho', 'globe', 'partisan', 'photographie', 'implant', 'apparent', 'portable', 'base', 'passion', 'riche', 'racial', 'baptismal', 'suspension', 'panorama', 'tenable', 'immigrant', 'impression', 'adulte', 'migration', 'ovulation', 'silo', 'évidemment', 'iota', 'observation', 'tango', 'explicable', 'narration', 'instrumentation', 'inadmissible', 'partition', 'corruption', 'suggestion', 'diode', 'omnivore', 'spectacle', 'classifier', 'compact', 'aberrant', 'via', 'tact', 'vocal', 'substitution', 'mauve', 'horde', 'incarnation', 'prude', 'ozone \xa0', 'shrapnel', 'solidification', 'harem', 'satin', 'accent', 'magnifique', 'collaboration', 'coagulation', 'divan', 'adorer', 'diamant', 'inséparable', 'cascade', 'incorrect', 'pH', 'testament', 'naval', 'admirable', 'constriction', 'oblong', 'cottage', 'difficulté', 'stimulation', 'recyclable', 'samba', 'ambivalent', 'leader', 'artistique', 'barracuda', 'cactus', 'lasso', 'perfec

In [8]:
pd.set_option("max_colwidth", None)
Sentences.head(10)

,Text,Difficulty
0,Pour moi cétait un cadeau de rêve,A1
1,A Barcelone mes parents ont réservé un appartement près de la plage,A1
2,À quelle heure on part,A1
3,Ainsi Pierre a le privilège dadmirer chaque jour lun des monuments les plus visités au monde,A1
4,Allumez votre téléphone mobile,A1
5,Après avoir bu une tasse de thé dans un café je rentre chez moi en bus,A1
6,Après je vais dans la cuisine et je bois mon café,A1
7,Arthur lace ses chaussures à rayures,A1
8,Aujourd’hui avec mes parents et ma sœur nous partons en voyage,A1
9,Aujourd’hui on est samedi nous rendons visite à notre grandmère,A1


In [9]:
#Calculating base rate (perfect base rate with 6 features is 0.16)
max(len(Sentences[Sentences["Difficulty"] =='A1']) / len(Sentences), len(Sentences[Sentences["Difficulty"] == 'A2']) / len(Sentences), len(Sentences[Sentences["Difficulty"] == 'B1']) / len(Sentences) , len(Sentences[Sentences["Difficulty"] == 'B2']) / len(Sentences),len(Sentences[Sentences["Difficulty"] == 'C1']) / len(Sentences), len(Sentences[Sentences["Difficulty"] == 'C2']) / len(Sentences))

0.18055555555555555

In [10]:
#Counting targets values 
display(Sentences.Difficulty.value_counts())

A1    221
A2    209
B2    203
C2    202
C1    197
B1    192
Name: Difficulty, dtype: int64

In [11]:
sp = spacy.load('fr_core_news_sm')

In [16]:
data = list(zip(Sentences["Text"],Sentences["Difficulty"]))
datawords = []
for t, label in data:
  list_of_strings = sp(t)
  token = [i.text for i in list_of_strings]
  datawords.append([token, label])
print(datawords)

[[['Pour', 'moi', 'cétait', 'un', 'cadeau', 'de', 'rêve'], 'A1'], [['A', 'Barcelone', 'mes', 'parents', 'ont', 'réservé', 'un', 'appartement', 'près', 'de', 'la', 'plage'], 'A1'], [['À', 'quelle', 'heure', 'on', 'part'], 'A1'], [['Ainsi', 'Pierre', 'a', 'le', 'privilège', 'dadmirer', 'chaque', 'jour', 'lun', 'des', 'monuments', 'les', 'plus', 'visités', 'au', 'monde'], 'A1'], [['Allumez', 'votre', 'téléphone', 'mobile'], 'A1'], [['Après', 'avoir', 'bu', 'une', 'tasse', 'de', 'thé', 'dans', 'un', 'café', 'je', 'rentre', 'chez', 'moi', 'en', 'bus'], 'A1'], [['Après', 'je', 'vais', 'dans', 'la', 'cuisine', 'et', 'je', 'bois', 'mon', 'café'], 'A1'], [['Arthur', 'lace', 'ses', 'chaussures', 'à', 'rayures'], 'A1'], [['Aujourd’hui', 'avec', 'mes', 'parents', 'et', 'ma', 'sœur', 'nous', 'partons', 'en', 'voyage'], 'A1'], [['Aujourd’hui', 'on', 'est', 'samedi', 'nous', 'rendons', 'visite', 'à', 'notre', 'grandmère'], 'A1'], [['Avec', 'le', 'printemps', 'qui', 'commence', 'en', 'mars', 'le', 'so

In [17]:
newdf = pd.DataFrame(datawords, columns=["Text", "Label"])

In [18]:
newdf.head(20)

,Text,Label
0,"[Pour, moi, cétait, un, cadeau, de, rêve]",A1
1,"[A, Barcelone, mes, parents, ont, réservé, un, appartement, près, de, la, plage]",A1
2,"[À, quelle, heure, on, part]",A1
3,"[Ainsi, Pierre, a, le, privilège, dadmirer, chaque, jour, lun, des, monuments, les, plus, visités, au, monde]",A1
4,"[Allumez, votre, téléphone, mobile]",A1
5,"[Après, avoir, bu, une, tasse, de, thé, dans, un, café, je, rentre, chez, moi, en, bus]",A1
6,"[Après, je, vais, dans, la, cuisine, et, je, bois, mon, café]",A1
7,"[Arthur, lace, ses, chaussures, à, rayures]",A1
8,"[Aujourd’hui, avec, mes, parents, et, ma, sœur, nous, partons, en, voyage]",A1
9,"[Aujourd’hui, on, est, samedi, nous, rendons, visite, à, notre, grandmère]",A1


In [19]:
#create NumberOfCognate and Cognates columns
newdf = newdf.assign(NumberOfCognate = 0) 
newdf = newdf.assign(Cognates = "")
newdf.head() 

,Text,Label,NumberOfCognate,Cognates
0,"[Pour, moi, cétait, un, cadeau, de, rêve]",A1,0,
1,"[A, Barcelone, mes, parents, ont, réservé, un, appartement, près, de, la, plage]",A1,0,
2,"[À, quelle, heure, on, part]",A1,0,
3,"[Ainsi, Pierre, a, le, privilège, dadmirer, chaque, jour, lun, des, monuments, les, plus, visités, au, monde]",A1,0,
4,"[Allumez, votre, téléphone, mobile]",A1,0,


In [20]:
index = 0
for sentence in newdf.Text:
  score=0
  liste =[]
  for word in sentence:
    if str(word) in ListCognates:
      score+=1
      liste.append(word)
  if index <=1223:
    newdf.at[index,'NumberOfCognate']= int(score)
    newdf.at[index,'Cognates']= np.array(liste)
    index += 1


In [21]:
newdf.head()

,Text,Label,NumberOfCognate,Cognates
0,"[Pour, moi, cétait, un, cadeau, de, rêve]",A1,0,[]
1,"[A, Barcelone, mes, parents, ont, réservé, un, appartement, près, de, la, plage]",A1,0,[]
2,"[À, quelle, heure, on, part]",A1,2,"[heure, part]"
3,"[Ainsi, Pierre, a, le, privilège, dadmirer, chaque, jour, lun, des, monuments, les, plus, visités, au, monde]",A1,1,[plus]
4,"[Allumez, votre, téléphone, mobile]",A1,2,"[téléphone, mobile]"


In [22]:
cognatesSum = newdf.NumberOfCognate.sum()
print("We found in total : ", cognatesSum, "cognates.")
print("The sentence with the most cognates is :")
newdf.sort_values(by=['NumberOfCognate'],ascending=False).head(1)

We found in total :  888 cognates.
The sentence with the most cognates is :


,Text,Label,NumberOfCognate,Cognates
1106,"[Jean, Buridan, philosophe, français, du, quatorzième, siècle, aurait, selon, la, légende, conçu, une, expérience, imaginaire, afin, de, prouver, l’, existence, du, libre, arbitre, , la, situation, serait, celle, d’, un, animal, en, l’, occurrence, un, âne, ayant, également, faim, et, soif, et, qui, placé, à, égale, distance, d’, une, botte, de, foin, et, d’, un, seau, d’, eau, hésite, se, montre, incapable, de, choisir, et, finalement, se, laisse, mourir]",C2,9,"[expérience, imaginaire, existence, situation, animal, occurrence, distance, incapable, finalement]"


In [23]:
del newdf['NumberOfCognate']

newdf

,Text,Label,Cognates
0,"[Pour, moi, cétait, un, cadeau, de, rêve]",A1,[]
1,"[A, Barcelone, mes, parents, ont, réservé, un, appartement, près, de, la, plage]",A1,[]
2,"[À, quelle, heure, on, part]",A1,"[heure, part]"
3,"[Ainsi, Pierre, a, le, privilège, dadmirer, chaque, jour, lun, des, monuments, les, plus, visités, au, monde]",A1,[plus]
4,"[Allumez, votre, téléphone, mobile]",A1,"[téléphone, mobile]"
...,...,...,...
1219,"[Les, enfants, chantèrent, les, chansons, qu’, ils, eurent, apprises, de, leur, professeur]",C2,[]
1220,"[Je, lus, un, roman, que, je, me, fus, procuré, à, la, bibliothèque, centrale]",C2,[]
1221,"[Vous, attendîtes, la, surprise, que, vos, parents, vous, eurent, promise]",C2,[surprise]
1222,"[Tu, portas, cette, robe, de, soie, bleue, que, j’, eus, cousue]",C2,[]


In [24]:
newdf = newdf[['Text', 'Cognates', 'Label']]
newdf

,Text,Cognates,Label
0,"[Pour, moi, cétait, un, cadeau, de, rêve]",[],A1
1,"[A, Barcelone, mes, parents, ont, réservé, un, appartement, près, de, la, plage]",[],A1
2,"[À, quelle, heure, on, part]","[heure, part]",A1
3,"[Ainsi, Pierre, a, le, privilège, dadmirer, chaque, jour, lun, des, monuments, les, plus, visités, au, monde]",[plus],A1
4,"[Allumez, votre, téléphone, mobile]","[téléphone, mobile]",A1
...,...,...,...
1219,"[Les, enfants, chantèrent, les, chansons, qu’, ils, eurent, apprises, de, leur, professeur]",[],C2
1220,"[Je, lus, un, roman, que, je, me, fus, procuré, à, la, bibliothèque, centrale]",[],C2
1221,"[Vous, attendîtes, la, surprise, que, vos, parents, vous, eurent, promise]",[surprise],C2
1222,"[Tu, portas, cette, robe, de, soie, bleue, que, j’, eus, cousue]",[],C2


In [28]:
df=pd.DataFrame()
df['Text'] = Sentences["Text"] + ' '+ newdf['Text'].astype(str) + ' ' + newdf['Cognates'].astype(str)
df['label'] = newdf['Label']
df

,Text,label
0,"Pour moi cétait un cadeau de rêve ['Pour', 'moi', 'cétait', 'un', 'cadeau', 'de', 'rêve'] []",A1
1,"A Barcelone mes parents ont réservé un appartement près de la plage ['A', 'Barcelone', 'mes', 'parents', 'ont', 'réservé', 'un', 'appartement', 'près', 'de', 'la', 'plage'] []",A1
2,"À quelle heure on part ['À', 'quelle', 'heure', 'on', 'part'] ['heure' 'part']",A1
3,"Ainsi Pierre a le privilège dadmirer chaque jour lun des monuments les plus visités au monde ['Ainsi', 'Pierre', 'a', 'le', 'privilège', 'dadmirer', 'chaque', 'jour', 'lun', 'des', 'monuments', 'les', 'plus', 'visités', 'au', 'monde'] ['plus']",A1
4,"Allumez votre téléphone mobile ['Allumez', 'votre', 'téléphone', 'mobile'] ['téléphone' 'mobile']",A1
...,...,...
1219,"Les enfants chantèrent les chansons qu’ils eurent apprises de leur professeur ['Les', 'enfants', 'chantèrent', 'les', 'chansons', 'qu’', 'ils', 'eurent', 'apprises', 'de', 'leur', 'professeur'] []",C2
1220,"Je lus un roman que je me fus procuré à la bibliothèque centrale ['Je', 'lus', 'un', 'roman', 'que', 'je', 'me', 'fus', 'procuré', 'à', 'la', 'bibliothèque', 'centrale'] []",C2
1221,"Vous attendîtes la surprise que vos parents vous eurent promise ['Vous', 'attendîtes', 'la', 'surprise', 'que', 'vos', 'parents', 'vous', 'eurent', 'promise'] ['surprise']",C2
1222,"Tu portas cette robe de soie bleue que j’eus cousue ['Tu', 'portas', 'cette', 'robe', 'de', 'soie', 'bleue', 'que', 'j’', 'eus', 'cousue'] []",C2


In [29]:
# Iformation about our dataset
display(df.shape, df.info(), df.describe)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1224 entries, 0 to 1223
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    1224 non-null   object
 1   label   1224 non-null   object
dtypes: object(2)
memory usage: 19.2+ KB


(1224, 2)

None

<bound method NDFrame.describe of                                                                                                                                                                                                                                                      Text label
0                                                                                                                                                            Pour moi cétait un cadeau de rêve ['Pour', 'moi', 'cétait', 'un', 'cadeau', 'de', 'rêve'] []    A1
1                                                                         A Barcelone mes parents ont réservé un appartement près de la plage ['A', 'Barcelone', 'mes', 'parents', 'ont', 'réservé', 'un', 'appartement', 'près', 'de', 'la', 'plage'] []    A1
2                                                                                                                                                                          À quelle heure on part ['À'

In [30]:
df.to_csv('Data_Tokenizer_cognates.csv', encoding='utf-8-sig', index=False)